In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')

C:\Users\USER\AppData\Local\Temp\ipykernel_17504\2964136533.py:1: ParserWarning: Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')
C:\Users\USER\AppData\Local\Temp\ipykernel_17504\2964136533.py:1: ParserWarning: Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')
C:\Users\USER\AppData\Local\Temp\ipykernel_17504\2964136533.py:1: ParserWarning: Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep

In [4]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [6]:
books = books[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "Image-URL-L"]]

In [7]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [8]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "publication_year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"
}, inplace=True)

In [9]:
books.head(2)

,ISBN,title,author,publication_year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [10]:
users = pd.read_csv("../data/BX-Users.csv", sep=";", encoding="latin-1", on_bad_lines='warn')

In [11]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
rating = pd.read_csv("../data/BX-Book-Ratings.csv", sep=";", encoding='latin-1', on_bad_lines='warn')

In [13]:
rating.head()
# rating.shape

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [14]:
rating.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating",
}, inplace=True)

In [15]:
print(books.shape)
print(users.shape)
print(rating.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [16]:
rating['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

## showing how many of the users give 200 + rating

In [17]:
x = rating['user_id'].value_counts() > 200
# before
print("before: ",x.shape) # 10Lakh plus
#after
print("after: ",x[x].shape) # only 899

before:  (105283,)
after:  (899,)


In [41]:
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [18]:
#storing the users index who rate 200+

y = x[x].index

In [20]:
ratings = rating[rating['user_id'].isin(y)]

In [21]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [22]:
rating_with_books = ratings.merge(books, on="ISBN")

In [42]:
rating_with_books.head(2)
# rating_with_books.shape

,user_id,ISBN,rating,title,author,publication_year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [24]:
books_num_rating = rating_with_books.groupby('title')['rating'].count().reset_index()

In [25]:
books_num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [26]:
books_num_rating.rename(columns={
    "rating": "number_of_ratings"
}, inplace=True)

In [27]:
# books_num_rating.head()
books_num_rating.shape

(160269, 2)

In [28]:
final_rating = rating_with_books.merge(books_num_rating, on="title")

In [30]:
final_rating.head(5)

# print(final_rating.shape)

,user_id,ISBN,rating,title,author,publication_year,publisher,image_url,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


In [31]:
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

In [32]:
final_rating.shape

(61853, 9)

In [33]:
final_rating.drop_duplicates(["user_id", "title"], inplace=True)

## final Rating after filtering and deleting all the duplicates ##

In [57]:
# final_rating.shape
final_rating

,user_id,ISBN,rating,title,author,publication_year,publisher,image_url,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79
...,...,...,...,...,...,...,...,...,...
487505,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,84
487506,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031362.0...,60
487579,275970,1573229725,0,Fingersmith,Sarah Waters,2002,Riverhead Books,http://images.amazon.com/images/P/1573229725.0...,59
487618,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris,2001,Time Warner Audio Major,http://images.amazon.com/images/P/1586210661.0...,146


## creating Pivot Table ##

In [43]:
book_pivot_table = final_rating.pivot_table(columns='user_id', index="title", values= "rating")

In [44]:
book_pivot_table

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [45]:
book_pivot_table.shape

(742, 888)

In [46]:
book_pivot_table.fillna(0, inplace=True)

In [47]:
book_pivot_table

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
from scipy.sparse import csr_matrix  ## import this to only use those fields who have value and nullify the 0 values

In [49]:
book_sparse = csr_matrix(book_pivot_table)

In [50]:
book_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14961 stored elements and shape (742, 888)>

In [52]:
from sklearn.neighbors import NearestNeighbors

In [53]:
book_model = NearestNeighbors(algorithm='brute')

In [54]:
book_model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [65]:
book_pivot_table.index[184] ## this is how we see each book index

'Exclusive'

In [56]:
len(book_pivot_table)

742

In [62]:
distance, suggesion = book_model.kneighbors(book_pivot_table.iloc[5, :].values.reshape(1,-1), n_neighbors=5)

In [63]:
distance

array([[ 0.        , 38.50973903, 38.92300091, 38.94868419, 39.1535439 ]])

In [64]:
suggesion

array([[  5, 184, 536, 372, 196]], dtype=int64)

In [66]:
for i in range(len(suggesion)):
    print(book_pivot_table.index[suggesion[i]])

Index(['A Bend in the Road', 'Exclusive', 'The Cradle Will Fall',
       'No Safe Place', 'Family Album'],
      dtype='object', name='title')


In [67]:
# book_pivot_table.index[741]
book_pivot_table.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [68]:
all_book_name = book_pivot_table.index

In [69]:
all_book_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [70]:
import pickle

pickle.dump(book_model,open("../Models/Book_Recc_Model.pkl", 'wb')) #creating our model || wb => write banner
pickle.dump(all_book_name,open("../Models/all_book_name.pkl", 'wb'))
pickle.dump(final_rating,open("../Models/final_ratings.pkl", 'wb'))
with open("../Models/book_pivot.pkl", 'wb') as f:
    pickle.dump(book_pivot_table, f)

In [71]:
def recommendedBook():
    book_name = input("Enter your book name: ").strip()
    num_of_sugg = int(input("Number of suggestions: ").strip())
    
    # Ensure book_pivot_table has a lowercased index for comparison
    lower_index = book_pivot_table.index.str.lower()
    
    # Find the book_id using the lowercased index

    try:
        book_id = np.where(lower_index == book_name.lower())[0][0]
    except IndexError:
        print("Book not found. Please try another book name.")
        return

    # Retrieve the nearest neighbors
    distance, suggestion = book_model.kneighbors(book_pivot_table.iloc[book_id, :].values.reshape(1, -1), n_neighbors=num_of_sugg)
    
    print("Recommended books:")
    for i in range(len(suggestion[0])):
        print(book_pivot_table.index[suggestion[0][i]])

# Example of how to use the function
# recommendedBook()


 ## demo of the recommended function

In [72]:
recommendedBook()

Recommended books:
Family Album
No Safe Place
Exclusive
The Cradle Will Fall
Table For Two
Fatal Cure
Fine Things
Deck the Halls (Holiday Classics)
Last Man Standing
Long After Midnight
